# CMDA 3654 Project - Money Ball 1

Marcos Fassio Bazzi - marcosfassiob18

Peter Do - kyungwan

Grant Collier - gcollier24

Ajay Kanjoor - ajkanjoor03

# Question: Which MLB teams will make the 2023 World Series?

Our goal for this project is to try to predict the 2023 MLB regular season and determine who will make the World Series based off of the data we can find [Baseball Reference](https://www.baseball-reference.com/) and [Spotrac](https://www.spotrac.com/mlb/payroll/2022/), two websites dedicated to baseball statistics and payroll, respectively.

## How does the MLB Regular Season work and how do teams make the World Series?

Major League Baseball is split up into two leagues: National League and American League. Each league is split up into 3 divisions: East, Central, and West. And each division has 5 teams in it making a total of 30 teams. Each team plays the four other teams in their division 19 times, and the 10 other teams in their league but not in their division either 6 or 7 times. This totals to a 162 game season for every team. The playoffs have changed throughout the years, but this year a total of 12 teams make the playoffs, 6 from both leagues. To win your division you need to have the best record out of the five teams, and the winners of each of the 6 division make the playoffs. In addition, out of all the teams that didn't win their division, the 3 teams with the best records in both leagues make the playoffs. In the playoffs, the two teams that win their league make it to the World Series. The 1 and 2 seed from both leagues get a by and automatically make it to the quarter finals, so those teams have an advantage.

# Scraping, cleaning, grouping and visualizing the data

Exporting the data from Baseball Reference is easy enough - the site has an option that allows us to share and export as a `.csv` file. However, the same cannot be said for Spotrac. In this case, we used Python libraries `requests` and `BeautifulSoup4` to fetch and scrape the payroll data from Spotrac.

## Importing necessary libraries

In [1]:
import requests
import bs4
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

## Scraping and cleaning the 2022 payroll data

In [2]:
# fetch link and get page
url = "https://www.spotrac.com/mlb/payroll/2022/"
header = { "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36" }
link = requests.get(url, headers=header)
soup = bs4.BeautifulSoup(link.content, "html5lib")

## get table and parse headers
table_data = soup.find('table', { "class": "datatable" })
headers = [i.text.strip() for i in table_data.find_all("th")]

## parse table body
table_body = soup.find("tbody")
team_data = []
for row_data in table_body.find_all("tr"):
    team_data += [[entry.text.strip().replace("\n", "").replace("\t", "") for entry in row_data.find_all("td")]]

# clean team names
for i in range(len(team_data)):
    team_data[i][1] = team_data[i][1][:-3] # last three characters are the team's abbreviation - we don't want this
team_data[4][1] = "San Diego Padres" # for whatever reason this entry's abbreviation was two characters instead of three
team_data.pop(14)
team_data.pop(14) # get rid of league averages in table

# turn data into dataframe
dirty_payroll_data_2022 = pd.DataFrame(team_data, columns=headers)

# cleaning 2022 payroll data
payroll_data_2022 = dirty_payroll_data_2022.copy()
payroll_data_2022 = dirty_payroll_data_2022.replace('0-','0').replace('\$','',regex=True).replace(',','', regex=True)
payroll_data_2022 = payroll_data_2022.loc[:, ['Team', '2022 Total Payroll']]
payroll_data_2022 = payroll_data_2022.astype({ '2022 Total Payroll':'int' })
payroll_data_2022.head()


,Rank,Team,Win%,Roster,26-Man Payroll,Injured Reserve,Retained,Buried,Suspended,2022 Total Payroll
0,1,Los Angeles Dodgers,0.685,28,231003922,28920266,5332866,7040000,35333333,270381426
1,2,New York Mets,0.623,31,231062889,4155954,28130999,14468000,0,268292506
2,3,New York Yankees,0.611,30,214676803,25203665,12115094,9307500,0,252957200
3,4,Philadelphia Phillies,0.537,28,191757795,11897332,30945214,17478000,0,244484097
4,5,San Diego Padres,0.549,29,173934032,10925000,34528348,6304700,5714286,224511694


## Scraping and cleaning the 2023 payroll data

The process is identical.

In [13]:
# fetch link and get page
url = "https://www.spotrac.com/mlb/payroll/2023/"
header = { "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36" }
link = requests.get(url, headers=header)
soup = bs4.BeautifulSoup(link.content, "html5lib")

## get table and parse headers
table_data = soup.find('table', { "class": "datatable" })
headers = [i.text.strip() for i in table_data.find_all("th")]

## parse table body
table_body = soup.find("tbody")
team_data = []
for row_data in table_body.find_all("tr"):
    team_data += [[entry.text.strip().replace("\n", "").replace("\t", "") for entry in row_data.find_all("td")]]

# clean team names
for i in range(len(team_data)):
    team_data[i][1] = team_data[i][1][:-3] # last three characters are the team's abbreviation - we don't want this
team_data[4][1] = "San Diego Padres" # for whatever reason this entry's abbreviation was two characters instead of three
team_data.pop(14)# get rid of league averages in table
team_data.pop(14)
team_data.pop(14)
team_data.pop(14)

# turn data into dataframe
dirty_payroll_data_2023 = pd.DataFrame(team_data, columns=headers)

# cleaning 2023 payroll data
payroll_data_2023 = dirty_payroll_data_2023.copy()
payroll_data_2023 = dirty_payroll_data_2023.replace('0-','0').replace('\$','',regex=True).replace(',','', regex=True)
payroll_data_2023 = payroll_data_2023.loc[:, ['Team', '2023 Total Payroll']]
payroll_data_2023 = payroll_data_2023.astype({ '2023 Total Payroll':'int' })

payroll_data_2023


,Team,2023 Total Payroll
0,New York Mets,344663493
1,New York Yankees,277033635
2,San Diego Padre,245117824
3,Philadelphia Phillies,242757097
4,San Diego Padres,224935134
5,Los Angeles Angels,212228094
6,Toronto Blue Jays,211000610
7,Atlanta Braves,201127608
8,Texas Rangers,195869490
9,Houston Astros,190417233


## Fetching the 2022 MLB team data from Baseball Reference

First, we copied the data as a `.csv` file and exported it. Then, we dropped the league average data from the bottom two indexes and renamed the "Tm" column to "Team" to group the data more easily.

In [ ]:
batting_data_2022_raw = pd.read_csv("data/batting_data_2022.csv")
pitching_data_2022_raw = pd.read_csv("data/pitching_data_2022.csv")

In [ ]:
batting_data_2022 = batting_data_2022_raw.drop([30, 31]).rename(columns={ "Tm": "Team" })
pitching_data_2022 = pitching_data_2022_raw.drop([30, 31]).rename(columns={ "Tm": "Team" })

batting_data_2022.head()

,Team,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB
0,Arizona Diamondbacks,57,26.5,4.33,162,6027,5351,702,1232,262,...,0.385,0.689,95,2061,97,60,31,50,14,1039
1,Atlanta Braves,53,27.5,4.87,162,6082,5509,789,1394,298,...,0.443,0.761,111,2443,103,66,1,36,13,1030
2,Baltimore Orioles,58,27.0,4.16,162,6049,5429,674,1281,275,...,0.390,0.695,97,2119,95,83,12,43,10,1095
3,Boston Red Sox,54,28.8,4.54,162,6144,5539,735,1427,352,...,0.409,0.731,102,2268,131,63,12,50,23,1133
4,Chicago Cubs,64,27.9,4.06,162,6072,5425,657,1293,265,...,0.387,0.698,96,2097,130,84,19,36,16,1100


## Grouping the 2022 payroll and baseball data

After cleaning the data above, we grouped the data together using two many-to-one joins based on the team name.

In [ ]:
baseball_data_2022 = pd.merge(batting_data_2022, pitching_data_2022, on=["Team"]) # merge baseball reference stats
baseball_data_2022 = pd.merge(baseball_data_2022, payroll_data_2022, on=["Team"]) # merge payroll stats

baseball_data_2022.head()

,Team,#Bat,BatAge,R/G,G_x,PA,AB,R_x,H_x,2B,...,LOB_y,Rank,Win%,Roster,26-Man Payroll,Injured Reserve,Retained,Buried,Suspended,2022 Total Payroll
0,Arizona Diamondbacks,57,26.5,4.33,162,6027,5351,702,1232,262,...,1051,25,0.457,28,53176803,19224958,9533407,9395200,0,85964090
1,Atlanta Braves,53,27.5,4.87,162,6082,5509,789,1394,298,...,1101,9,0.623,28,143633732,24418726,10710954,11610000,0,183438888
2,Baltimore Orioles,58,27.0,4.16,162,6049,5429,674,1281,275,...,1092,30,0.512,29,22538146,5709974,13258460,10540500,0,44888388
3,Boston Red Sox,54,28.8,4.54,162,6144,5539,735,1427,352,...,1109,6,0.481,25,109769156,50192677,48696520,27500000,0,211812131
4,Chicago Cubs,64,27.9,4.06,162,6072,5425,657,1293,265,...,1130,14,0.457,28,83228504,44219852,20225459,9707000,0,151054737
